In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report , accuracy_score
from sklearn.model_selection import train_test_split
import math
import csv

In [103]:
np.random.seed(42)
random.seed(42)

In [104]:
data = pd.read_csv('processed.cleveland.data' , header=None)
data.shape


(303, 14)

In [105]:
all_columns = ['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal','target']
data.columns = all_columns
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


In [106]:
data = data[~data.isin(['?']).any(axis=1)]
print(data.shape)
data['target'].value_counts()

(297, 14)


target
0    160
1     54
2     35
3     35
4     13
Name: count, dtype: int64

In [107]:
data['target'] = data['target'].replace(0, 0)
data['target'] = data['target'].replace(1, 1)
data['target'] = data['target'].replace(2, 2)
data['target'] = data['target'].replace(3, 2)
data['target'] = data['target'].replace(4, 1)

In [108]:
data = data[data['target'].isin([1,2])]

In [109]:
data

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
6,62.0,0.0,4.0,140.0,268.0,0.0,2.0,160.0,0.0,3.6,3.0,2.0,3.0,2
8,63.0,1.0,4.0,130.0,254.0,0.0,2.0,147.0,0.0,1.4,2.0,1.0,7.0,2
9,53.0,1.0,4.0,140.0,203.0,1.0,2.0,155.0,1.0,3.1,3.0,0.0,7.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,57.0,0.0,4.0,140.0,241.0,0.0,0.0,123.0,1.0,0.2,2.0,0.0,7.0,1
298,45.0,1.0,1.0,110.0,264.0,0.0,0.0,132.0,0.0,1.2,2.0,0.0,7.0,1
299,68.0,1.0,4.0,144.0,193.0,1.0,0.0,141.0,0.0,3.4,2.0,2.0,7.0,2
300,57.0,1.0,4.0,130.0,131.0,0.0,0.0,115.0,1.0,1.2,2.0,1.0,7.0,2


In [110]:
data['target'].value_counts()

target
2    70
1    67
Name: count, dtype: int64

In [ ]:
def scaling(x_train,x_test):
  columns = x_train.columns
  scaler = StandardScaler()
  x_train = scaler.fit_transform(x_train)
  x_test = scaler.transform(x_test)

  x_train = pd.DataFrame(x_train,columns=columns)
  x_test = pd.DataFrame(x_test,columns=columns)

  return x_train,x_test

def data_split(data , features):
  x = data[features]
  y = data['target']

  x_train , x_test , y_train , y_test = train_test_split(x,y,test_size=0.2,random_state=42,stratify=y)
  return x_train , x_test , y_train , y_test


In [112]:
x_train , x_test , y_train , y_test = data_split(data,['trestbps', 'slope', 'restecg', 'sex', 'fbs', 'thal', 'age', 'oldpeak'])
x_train,x_test = scaling(x_train,x_test)

In [113]:
import joblib
model = joblib.load('model_14_23.pkl')
y_pred = model.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.86      0.86      0.86        14
           2       0.86      0.86      0.86        14

    accuracy                           0.86        28
   macro avg       0.86      0.86      0.86        28
weighted avg       0.86      0.86      0.86        28



In [ ]:
classifier = RandomForestClassifier(n_estimators=10,max_depth=5,random_state=42)
classifier.fit(x_train,y_train)
y_pred_test = classifier.predict(x_test)
y_pred_train = classifier.predict(x_train)
print(classification_report(y_test,y_pred_test))


In [406]:
# import joblib
# joblib.dump(classifier, 'model_14_23.pkl')

['model_14_23.pkl']

In [ ]:
# def pipeline_1_4(no_of_features, all_columns, data, n_estimators=100, max_depth=None):
#     """Pipeline for classifying between classes 2 and 3"""
#     features = all_columns.copy()
#     features.remove('target')
#     random.shuffle(features)
    
#     selected_features = features[:no_of_features]
#     x_train, x_test, y_train, y_test = data_split(data, selected_features)
#     x_train, x_test = scaling(x_train, x_test)
    
#     classifier = RandomForestClassifier(
#         n_estimators=n_estimators,
#         max_depth=max_depth,
#         random_state=42,
#     )
#     classifier.fit(x_train, y_train)
#     y_test_pred = classifier.predict(x_test)
#     y_train_pred = classifier.predict(x_train)
    
#     return (
#         accuracy_score(y_test, y_test_pred),
#         accuracy_score(y_train, y_train_pred),
#         selected_features
#     )

# # Filter data for only classes 2 and 3
# data_14_23 = data[data['target'].isin([1, 2])]

# # Create and write to CSV file
# filename = "results_class_14_23.csv"
# header = ['n_estimators', 'max_depth', 'no_of_features','train_accuracy', 'test_accuracy', 'selected_features']

# with open(filename, mode='w', newline='') as file:
#     writer = csv.DictWriter(file, fieldnames=header)
#     writer.writeheader()

# with open(filename, mode='a', newline='') as file:
#     writer = csv.DictWriter(file, fieldnames=header)
    
#     # Grid search over parameters
#     for n_estimators in range(2, 20, 2):  # Step of 10
#         for max_depth in range(1, 8):      # 1 to 10
#             for num_features in range(1, len(all_columns)):
#                 for iteration in range(10):  # 20 iterations per combination
#                     test_acc, train_acc, selected_subset = pipeline_1_4(
#                         num_features,
#                         all_columns,
#                         data_14_23,
#                         n_estimators=n_estimators,
#                         max_depth=max_depth
#                     )
                    
#                     # Write results to CSV
#                     writer.writerow({
#                         'n_estimators': n_estimators,
#                         'max_depth': max_depth,
#                         'no_of_features': num_features,
#                         'train_accuracy': train_acc,
#                         'test_accuracy': test_acc,
#                         'selected_features': selected_subset
#                     })

# print(f"Results have been saved to {filename}")

Results have been saved to results_class_14_23.csv


In [4]:
import pandas as pd

# Read the CSV file
tryi = pd.read_csv('raw_results/results_class_14_23.csv')

# Filter rows where the difference between test and train accuracy is less than 0.05 (no underfitting)
filtered_df = tryi[(tryi['test_accuracy'] - tryi['train_accuracy']) < 0.05 ]

# From these, select the row(s) with the maximum test accuracy
max_test_acc_row = filtered_df[filtered_df['test_accuracy'] == filtered_df['test_accuracy'].max()]
max_test_acc_row.head()


,n_estimators,max_depth,no_of_features,train_accuracy,test_accuracy,selected_features
4022,10,3,13,0.825688,0.857143,"['cp', 'restecg', 'oldpeak', 'thal', 'trestbps..."
4232,10,5,8,0.853211,0.857143,"['trestbps', 'slope', 'restecg', 'sex', 'fbs',..."
4913,12,3,11,0.807339,0.857143,"['trestbps', 'restecg', 'oldpeak', 'fbs', 'sex..."
6090,14,5,12,0.908257,0.857143,"['ca', 'sex', 'oldpeak', 'trestbps', 'exang', ..."
6887,16,4,13,0.871560,0.857143,"['cp', 'fbs', 'slope', 'trestbps', 'restecg', ..."


In [ ]:
'''
for binary classification 
class name = 0 , 1
n_estimators = 41
max_depth = 5
no_of_features = 11
features = ['thal', 'exang', 'thalach', 'ca', 'chol', 'slope', 'fbs', 'sex', 'restecg', 'age', 'oldpeak']

                  precision    recall  f1-score   support

           0       0.86      0.97      0.91        32
           1       0.96      0.82      0.88        28

    accuracy                           0.90        60
   macro avg       0.91      0.90      0.90        60
weighted avg       0.91      0.90      0.90        60

_______________________________________________________________________________________________

for 14 vs 23 classification
class name = 1 (for 14) , 2 (for 23)
n_estimators = 10
max_depth = 5
no_of_features = 8 
features = ['trestbps', 'slope', 'restecg', 'sex', 'fbs', 'thal', 'age', 'oldpeak']

                  precision    recall  f1-score   support

           1       0.86      0.86      0.86        14
           2       0.86      0.86      0.86        14

    accuracy                           0.86        28
   macro avg       0.86      0.86      0.86        28
weighted avg       0.86      0.86      0.86        28
___________________________________________________________________________


for 1 vs 4 classification
class name = 1 , 4 
n_estimators = 2
max_depth = 4
no_of_features = 10
features = ['sex', 'slope', 'fbs', 'thalach', 'trestbps', 'restecg', 'thal', 'cp', 'oldpeak', 'exang']

                  precision    recall  f1-score   support

           1       0.92      1.00      0.96        11
           4       1.00      0.67      0.80         3

    accuracy                           0.93        14
   macro avg       0.96      0.83      0.88        14
weighted avg       0.93      0.93      0.92        14

_____________________________________________________________________________

for 2 vs 3 classification
class name = 2 , 3
n_estimators = 10
max_depth = 6
no_of_features = 3
features = ['trestbps', 'slope', 'oldpeak']

                precision    recall  f1-score   support

          2       0.78      1.00      0.88         7
          3       1.00      0.71      0.83         7

    accuracy                           0.86        14
   macro avg       0.89      0.86      0.85        14
weighted avg       0.89      0.86      0.85        14


'''

'\nfor binary classification \n'

In [ ]:
raw_models = {
    'binary': load('raw_models/model_binary.pkl'),
    '14_23': load('raw_models/model_14_23.pkl'),
    '1_4': load('raw_models/model_1_4.pkl'),
    '2_3': load('raw_models/model_2_3.pkl'),
}

raw_scalers = {
    'binary': load('raw_scaler/scaler_binary.pkl'),
    '14_23': load('raw_scaler/scaler_14_23.pkl'),
    '1_4': load('raw_scaler/scaler_1_4.pkl'),
    '2_3': load('raw_scaler/scaler_2_3.pkl'),
}

raw_features_dict = {
    'binary': ['thal', 'exang', 'thalach', 'ca', 'chol', 'slope', 'fbs', 'sex', 'restecg', 'age', 'oldpeak'],
    '1_4': ['sex', 'slope', 'fbs', 'thalach', 'trestbps', 'restecg', 'thal', 'cp', 'oldpeak', 'exang'],
    '2_3': ['trestbps', 'slope', 'oldpeak'],
    '14_23': ['trestbps', 'slope', 'restecg', 'sex', 'fbs', 'thal', 'age', 'oldpeak'],
}

